In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image, ImageDraw
import numpy as np
from tqdm import tqdm
import glob
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
IMG_SIZE = 416
BATCH_SIZE = 8
LEARNING_RATE = 1e-5 # Lower LR for a more complex model
EPOCHS = 50
NUM_CLASSES = 11
CONFIDENCE_THRESHOLD = 0.6
IOU_THRESHOLD = 0.5
ANCHORS = [
    [(116, 90), (156, 198), (373, 326)],  # Scale 1 (13x13) for large objects
    [(30, 61), (62, 45), (59, 119)],      # Scale 2 (26x26) for medium objects
    [(10, 13), (16, 30), (33, 23)],        # Scale 3 (52x52) for small objects
]
S = [IMG_SIZE // 32, IMG_SIZE // 16, IMG_SIZE // 8] # Strides, Grid sizes: [13, 26, 52]

In [3]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, use_bn=True, **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=not use_bn, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels) if use_bn else nn.Identity()
        self.leaky = nn.LeakyReLU(0.1)

    def forward(self, x):
        return self.leaky(self.bn(self.conv(x)))

class ResidualBlock(nn.Module):
    def __init__(self, channels, num_repeats=1):
        super().__init__()
        self.layers = nn.ModuleList()
        for _ in range(num_repeats):
            self.layers += [
                nn.Sequential(
                    CNNBlock(channels, channels // 2, kernel_size=1),
                    CNNBlock(channels // 2, channels, kernel_size=3, padding=1),
                )
            ]
        self.num_repeats = num_repeats

    def forward(self, x):
        for layer in self.layers:
            x = x + layer(x)
        return x

class PredictionHead(nn.Module):
    def __init__(self, in_channels, num_classes, anchors):
        super().__init__()
        self.num_classes = num_classes
        self.anchors = anchors
        self.num_anchors = len(anchors)

        self.head = nn.Sequential(
            CNNBlock(in_channels, in_channels * 2, kernel_size=3, padding=1),
            CNNBlock(in_channels * 2, (self.num_anchors * (5 + num_classes)), use_bn=False, kernel_size=1),
        )

    def forward(self, x):
        # Reshape the output to [Batch, Num_Anchors, Grid_S, Grid_S, 5 + Num_Classes]
        out = self.head(x)
        out = out.view(x.shape[0], self.num_anchors, 5 + self.num_classes, x.shape[2], x.shape[3])
        out = out.permute(0, 1, 3, 4, 2)
        return out

class Darknet53(nn.Module):
    def __init__(self, in_channels=1):
        super().__init__()
        self.in_channels = in_channels
        self.layers = nn.ModuleList([
            CNNBlock(in_channels, 32, kernel_size=3, padding=1),
            CNNBlock(32, 64, kernel_size=3, padding=1, stride=2),
            ResidualBlock(64, num_repeats=1),
            CNNBlock(64, 128, kernel_size=3, padding=1, stride=2),
            ResidualBlock(128, num_repeats=2),
            CNNBlock(128, 256, kernel_size=3, padding=1, stride=2),
            ResidualBlock(256, num_repeats=8), # -> Route 1
            CNNBlock(256, 512, kernel_size=3, padding=1, stride=2),
            ResidualBlock(512, num_repeats=8), # -> Route 2
            CNNBlock(512, 1024, kernel_size=3, padding=1, stride=2),
            ResidualBlock(1024, num_repeats=4),
        ])

    def forward(self, x):
        outputs = []
        for i, layer in enumerate(self.layers):
            x = layer(x)
            # The routes are the outputs of the last three residual blocks
            if i in [6, 8]:
                outputs.append(x)
        outputs.append(x)
        return outputs[0], outputs[1], outputs[2] # 52x52, 26x26, 13x13

In [4]:
# class YOLOv3(nn.Module):
#     def __init__(self, in_channels=1, num_classes=11):
#         super().__init__()
#         self.num_classes = num_classes
#         self.in_channels = in_channels
#         self.backbone = Darknet53(in_channels=in_channels)
        
#         # Prediction Heads for each scale
#         self.head1 = PredictionHead(1024, num_classes, ANCHORS[0]) # Large scale
#         self.head2 = PredictionHead(512, num_classes, ANCHORS[1])  # Medium scale
#         self.head3 = PredictionHead(256, num_classes, ANCHORS[2])  # Small scale

#         self.conv1 = CNNBlock(1024, 512, kernel_size=1)
#         self.conv2 = CNNBlock(512, 256, kernel_size=1)
#         self.upsample = nn.Upsample(scale_factor=2, mode="nearest")

#     def forward(self, x):
#         route3, route2, route1 = self.backbone(x) # small, medium, large routes
        
#         # Scale 1 prediction (large objects)
#         out1 = self.head1(route1)
        
#         # Scale 2 prediction (medium objects)
#         x = self.conv1(route1)
#         x = self.upsample(x)
#         x = torch.cat([x, route2], dim=1)
#         out2 = self.head2(x)

#         # Scale 3 prediction (small objects)
#         x = self.conv2(x)
#         x = self.upsample(x)
#         x = torch.cat([x, route3], dim=1)
#         out3 = self.head3(x)

#         return out1, out2, out3
    

class YOLOv3(nn.Module):
    def __init__(self, in_channels=1, num_classes=11):
        super().__init__()
        self.num_classes = num_classes
        self.in_channels = in_channels
        self.backbone = Darknet53(in_channels=in_channels)
        
        # Prediction Heads for each scale
        self.head1 = PredictionHead(1024, num_classes, ANCHORS[0]) # Large scale
        
        # --- FIX 1: Update head2 to accept 1024 channels ---
        # (512 from upsample + 512 from route2)
        self.head2 = PredictionHead(1024, num_classes, ANCHORS[1])  
        
        # --- FIX 2: Update head3 to accept 512 channels ---
        # (256 from upsample + 256 from route3)
        self.head3 = PredictionHead(512, num_classes, ANCHORS[2])  

        self.conv1 = CNNBlock(1024, 512, kernel_size=1)
        
        # --- FIX 3: Update conv2 to accept 1024 channels ---
        # It takes the output of Scale 2 concat (1024) and reduces it to 256
        self.conv2 = CNNBlock(1024, 256, kernel_size=1)
        
        self.upsample = nn.Upsample(scale_factor=2, mode="nearest")

    def forward(self, x):
        route3, route2, route1 = self.backbone(x) # small, medium, large routes
        
        # Scale 1 prediction (large objects)
        out1 = self.head1(route1)
        
        # Scale 2 prediction (medium objects)
        x = self.conv1(route1)
        x = self.upsample(x)
        x = torch.cat([x, route2], dim=1)
        out2 = self.head2(x) # Input is now 1024, matches head2

        # Scale 3 prediction (small objects)
        x = self.conv2(x)    # Input is 1024, matches conv2
        x = self.upsample(x)
        x = torch.cat([x, route3], dim=1)
        out3 = self.head3(x) # Input is now 512, matches head3

        return out1, out2, out3

In [5]:
def iou_boxes(box1, box2, box_format="xywh"):

    eps=1e-6

    if box_format == "xywh":
        box1_x1 = box1[..., 0:1] - box1[..., 2:3] / 2
        box1_y1 = box1[..., 1:2] - box1[..., 3:4] / 2
        box1_x2 = box1[..., 0:1] + box1[..., 2:3] / 2
        box1_y2 = box1[..., 1:2] + box1[..., 3:4] / 2

        box2_x1 = box2[..., 0:1] - box2[..., 2:3] / 2
        box2_y1 = box2[..., 1:2] - box2[..., 3:4] / 2
        box2_x2 = box2[..., 0:1] + box2[..., 2:3] / 2
        box2_y2 = box2[..., 1:2] + box2[..., 3:4] / 2
    else: # box_format == "xyxy"
        box1_x1, box1_y1, box1_x2, box1_y2 = box1[..., 0:1], box1[..., 1:2], box1[..., 2:3], box1[..., 3:4]
        box2_x1, box2_y1, box2_x2, box2_y2 = box2[..., 0:1], box2[..., 1:2], box2[..., 2:3], box2[..., 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    inter_w = (x2 - x1).clamp(min=0)
    inter_h = (y2 - y1).clamp(min=0)
    intersection = inter_w * inter_h 

    area1 = abs((box1_x2 - box1_x1).clamp(min=0) * (box1_y2 - box1_y1).clamp(min=0))
    area2 = abs((box2_x2 - box2_x1).clamp(min=0) * (box2_y2 - box2_y1).clamp(min=0))

    union = area1 + area2 - intersection

    iou=intersection / (union+eps)

    return iou

In [6]:
def non_max_suppression(bboxes, iou_threshold, confidence_threshold):
    # Filter out boxes with low confidence
    bboxes = [box for box in bboxes if box[1] > confidence_threshold]
    # Sort boxes by confidence score in descending order
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
    bboxes_after_nms = []

    while bboxes:
        chosen_box = bboxes.pop(0)
        
        # Keep only boxes of different classes or with low IoU
        bboxes = [
            box
            for box in bboxes
            if box[0] != chosen_box[0] or 
               iou_boxes(torch.tensor(chosen_box[2:]), torch.tensor(box[2:])) < iou_threshold
        ]
        
        bboxes_after_nms.append(chosen_box)

    return bboxes_after_nms

In [7]:
# class RadarDataset(Dataset):
#     def __init__(self, image_dir, label_dir, anchors, S, C=1):
#         self.image_paths = sorted(glob.glob(os.path.join(image_dir, '*.png')))
#         self.label_dir = label_dir
#         self.S = S
#         self.C = C
#         self.anchors = torch.tensor(anchors[0] + anchors[1] + anchors[2])
#         self.num_anchors = self.anchors.shape[0]
#         self.num_anchors_per_scale = self.num_anchors // 3
#         self.ignore_iou_thresh = 0.5
        
#         self.transform = transforms.Compose([
#             transforms.Grayscale(),
#             transforms.Resize((IMG_SIZE, IMG_SIZE)),
#             transforms.ToTensor()
#         ])

#     def __len__(self):
#         return len(self.image_paths)

#     # def __getitem__(self, idx):
#     #     image_path = self.image_paths[idx]
#     #     label_path = os.path.join(self.label_dir, os.path.basename(image_path).replace('.png', '.txt'))
#     #     image = Image.open(image_path)
#     #     image = self.transform(image)

#     #     targets = [torch.zeros((self.num_anchors_per_scale, s, s, 6)) for s in self.S]

#     #     if os.path.exists(label_path):
#     #         with open(label_path, 'r') as f:
#     #             for line in f:
#     #                 cls, x, y, w, h = map(float, line.strip().split())
                    
#     #                 # Find the best anchor for this bounding box across ALL anchors
#     #                 ious = iou_boxes(torch.tensor([w, h]), self.anchors)
#     #                 best_anchor_idx = ious.argmax()
                    
#     #                 # Determine which scale and which anchor on that scale it belongs to
#     #                 scale_idx = best_anchor_idx // self.num_anchors_per_scale
#     #                 anchor_on_scale_idx = best_anchor_idx % self.num_anchors_per_scale
                    
#     #                 s = self.S[scale_idx]
#     #                 i, j = int(s * y), int(s * x) # grid cell
                    
#     #                 # Check if cell is already taken
#     #                 if targets[scale_idx][anchor_on_scale_idx, i, j, 0] == 0:
#     #                     targets[scale_idx][anchor_on_scale_idx, i, j, 0] = 1 
#     #                     x_cell, y_cell = s * x - j, s * y - i
#     #                     w_cell, h_cell = w, h
#     #                     box_coords = torch.tensor([x_cell, y_cell, w_cell, h_cell])
#     #                     targets[scale_idx][anchor_on_scale_idx, i, j, 1:5] = box_coords
#     #                     targets[scale_idx][anchor_on_scale_idx, i, j, 5] = int(cls)

#     #     return image, tuple(targets)
    

#     def __getitem__(self, idx):
#         image_path = self.image_paths[idx]
#         label_path = os.path.join(self.label_dir, os.path.basename(image_path).replace('.png', '.txt'))
#         image = Image.open(image_path)
#         image = self.transform(image)

#         targets = [torch.zeros((self.num_anchors_per_scale, s, s, 6)) for s in self.S]

#         if os.path.exists(label_path):
#             with open(label_path, 'r') as f:
#                 for line in f:
#                     cls, x, y, w, h = map(float, line.strip().split())
                    
#                     # --- FIX STARTS HERE ---
#                     # We need to trick iou_boxes into comparing just width/height.
#                     # We create boxes centered at (0,0) with the given width/height.
                    
#                     # Create current box with x=0, y=0, w=w, h=h
#                     box_tensor = torch.tensor([0, 0, w, h])
                    
#                     # Create anchor boxes with x=0, y=0, w=anchor_w, h=anchor_h
#                     # self.anchors is (9, 2), we need (9, 4)
#                     anchors_xywh = torch.cat([torch.zeros_like(self.anchors), self.anchors], dim=1)
                    
#                     # Calculate IoU using the padded coordinates
#                     ious = iou_boxes(box_tensor, anchors_xywh)
#                     # --- FIX ENDS HERE ---

#                     best_anchor_idx = ious.argmax()
                    
#                     # Determine which scale and which anchor on that scale it belongs to
#                     scale_idx = best_anchor_idx // self.num_anchors_per_scale
#                     anchor_on_scale_idx = best_anchor_idx % self.num_anchors_per_scale
                    
#                     s = self.S[scale_idx]
#                     i, j = int(s * y), int(s * x) # grid cell
                    
#                     # Check if cell is already taken
#                     if targets[scale_idx][anchor_on_scale_idx, i, j, 0] == 0:
#                         targets[scale_idx][anchor_on_scale_idx, i, j, 0] = 1 
#                         x_cell, y_cell = s * x - j, s * y - i
#                         w_cell, h_cell = w, h
#                         box_coords = torch.tensor([x_cell, y_cell, w_cell, h_cell])
#                         targets[scale_idx][anchor_on_scale_idx, i, j, 1:5] = box_coords
#                         targets[scale_idx][anchor_on_scale_idx, i, j, 5] = int(cls)

#         return image, tuple(targets)


In [8]:
class RadarDataset(Dataset):
    def __init__(self, image_dir, label_dir, anchors, S, C=1):
        self.image_paths = sorted(glob.glob(os.path.join(image_dir, '*.png')))
        self.label_dir = label_dir
        self.S = S
        self.C = C
        self.anchors = torch.tensor(anchors[0] + anchors[1] + anchors[2])
        self.num_anchors = self.anchors.shape[0]
        self.num_anchors_per_scale = self.num_anchors // 3
        self.ignore_iou_thresh = 0.5
        
        self.transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label_path = os.path.join(self.label_dir, os.path.basename(image_path).replace('.png', '.txt'))
        image = Image.open(image_path)
        image = self.transform(image)

        targets = [torch.zeros((self.num_anchors_per_scale, s, s, 6)) for s in self.S]

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    cls, x, y, w, h = map(float, line.strip().split())
                    
                    # --- PREVIOUS FIX: Fake coordinates for IoU ---
                    box_tensor = torch.tensor([0, 0, w, h])
                    anchors_xywh = torch.cat([torch.zeros_like(self.anchors), self.anchors], dim=1)
                    ious = iou_boxes(box_tensor, anchors_xywh)
                    
                    best_anchor_idx = ious.argmax()
                    
                    scale_idx = best_anchor_idx // self.num_anchors_per_scale
                    anchor_on_scale_idx = best_anchor_idx % self.num_anchors_per_scale
                    
                    s = self.S[scale_idx]
                    
                    # --- NEW FIX STARTS HERE ---
                    # Calculate indices
                    i, j = int(s * y), int(s * x) 
                    
                    # Clamp indices to ensure they stay within valid bounds [0, s-1]
                    # This handles cases where x or y is exactly 1.0
                    i = min(i, s - 1)
                    j = min(j, s - 1)
                    # --- NEW FIX ENDS HERE ---
                    
                    if targets[scale_idx][anchor_on_scale_idx, i, j, 0] == 0:
                        targets[scale_idx][anchor_on_scale_idx, i, j, 0] = 1 
                        x_cell, y_cell = s * x - j, s * y - i
                        w_cell, h_cell = w, h
                        box_coords = torch.tensor([x_cell, y_cell, w_cell, h_cell])
                        targets[scale_idx][anchor_on_scale_idx, i, j, 1:5] = box_coords
                        targets[scale_idx][anchor_on_scale_idx, i, j, 5] = int(cls)

        return image, tuple(targets)

In [9]:
# class YoloLoss(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.mse = nn.MSELoss()
#         self.bce = nn.BCEWithLogitsLoss()
#         self.lambda_class = 1
#         self.lambda_noobj = 10
#         self.lambda_obj = 1
#         self.lambda_box = 10

#     def forward(self, predictions, targets, anchors):
#         total_loss = 0
        
#         # Iterate over the 3 scales
#         for i in range(3):
#             pred = predictions[i]
#             target = targets[i]
#             # Anchors for the current scale
#             scale_anchors = anchors[i]
            
#             obj_mask = target[..., 0] == 1
#             noobj_mask = target[..., 0] == 0

#             # No Object Loss
#             noobj_loss = self.bce(
#                 (pred[..., 0:1][noobj_mask]), (target[..., 0:1][noobj_mask])
#             )

#             # Object Loss
#             obj_loss = self.bce(
#                 (pred[..., 0:1][obj_mask]), (target[..., 0:1][obj_mask])
#             )

#             # Box Coordinate Loss
#             pred[..., 1:3] = torch.sigmoid(pred[..., 1:3]) # x,y
#             target[..., 3:5] = torch.log(
#                 (1e-16 + target[..., 3:5] / scale_anchors)
#             ) # w,h
#             box_loss = self.mse(pred[..., 1:5][obj_mask], target[..., 1:5][obj_mask])
            
#             # Class Loss
#             class_loss = self.bce(
#                 (pred[..., 5:][obj_mask]), (target[..., 5:][obj_mask].float())
#             )
            
#             total_loss += (
#                 self.lambda_box * box_loss
#                 + self.lambda_obj * obj_loss
#                 + self.lambda_noobj * noobj_loss
#                 + self.lambda_class * class_loss
#             )
            
#         return total_loss
    

# class YoloLoss(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.mse = nn.MSELoss()
#         self.bce = nn.BCEWithLogitsLoss()
#         self.lambda_class = 1
#         self.lambda_noobj = 10
#         self.lambda_obj = 1
#         self.lambda_box = 10

#     def forward(self, predictions, targets, anchors):
#         total_loss = 0
        
#         # Iterate over the 3 scales
#         for i in range(3):
#             pred = predictions[i]
#             target = targets[i]
            
#             # --- FIX STARTS HERE ---
#             # Reshape anchors to (1, 3, 1, 1, 2) to allow broadcasting
#             # against target shape (Batch, 3, Grid, Grid, 2)
#             scale_anchors = anchors[i].reshape(1, 3, 1, 1, 2)
#             # --- FIX ENDS HERE ---
            
#             obj_mask = target[..., 0] == 1
#             noobj_mask = target[..., 0] == 0

#             # No Object Loss
#             noobj_loss = self.bce(
#                 (pred[..., 0:1][noobj_mask]), (target[..., 0:1][noobj_mask])
#             )

#             # Object Loss
#             obj_loss = self.bce(
#                 (pred[..., 0:1][obj_mask]), (target[..., 0:1][obj_mask])
#             )

#             # Box Coordinate Loss
#             pred[..., 1:3] = torch.sigmoid(pred[..., 1:3]) # x,y
            
#             # Now this division works because dimensions align correctly
#             target[..., 3:5] = torch.log(
#                 (1e-16 + target[..., 3:5] / scale_anchors)
#             ) # w,h
            
#             box_loss = self.mse(pred[..., 1:5][obj_mask], target[..., 1:5][obj_mask])
            
#             # Class Loss
#             class_loss = self.bce(
#                 (pred[..., 5:][obj_mask]), (target[..., 5:][obj_mask].float())
#             )
            
#             total_loss += (
#                 self.lambda_box * box_loss
#                 + self.lambda_obj * obj_loss
#                 + self.lambda_noobj * noobj_loss
#                 + self.lambda_class * class_loss
#             )
            
#         return total_loss
    

# class YoloLoss(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.mse = nn.MSELoss()
#         self.bce = nn.BCEWithLogitsLoss()
#         self.lambda_class = 1
#         self.lambda_noobj = 10
#         self.lambda_obj = 1
#         self.lambda_box = 10

#     def forward(self, predictions, targets, anchors):
#         total_loss = 0
        
#         for i in range(3):
#             pred = predictions[i]
#             target = targets[i]
            
#             # Reshape anchors for broadcasting (from previous fix)
#             scale_anchors = anchors[i].reshape(1, 3, 1, 1, 2)
            
#             obj_mask = target[..., 0] == 1
#             noobj_mask = target[..., 0] == 0

#             # No Object Loss
#             noobj_loss = self.bce(
#                 (pred[..., 0:1][noobj_mask]), (target[..., 0:1][noobj_mask])
#             )

#             # Object Loss
#             obj_loss = self.bce(
#                 (pred[..., 0:1][obj_mask]), (target[..., 0:1][obj_mask])
#             )

#             # Box Coordinate Loss
#             pred[..., 1:3] = torch.sigmoid(pred[..., 1:3]) # x,y
#             target[..., 3:5] = torch.log(
#                 (1e-16 + target[..., 3:5] / scale_anchors)
#             ) # w,h
#             box_loss = self.mse(pred[..., 1:5][obj_mask], target[..., 1:5][obj_mask])
            
#             # --- FIX STARTS HERE (Class Loss) ---
#             if obj_mask.any():
#                 # predictions: (N_objects, num_classes)
#                 pred_classes = pred[..., 5:][obj_mask]
                
#                 # target indices: (N_objects) - select index 5
#                 target_indices = target[..., 5][obj_mask].long()
                
#                 # One-hot encode targets to match prediction shape (N, 11)
#                 num_classes = pred_classes.shape[1]
#                 target_one_hot = torch.nn.functional.one_hot(target_indices, num_classes=num_classes).float()
                
#                 class_loss = self.bce(pred_classes, target_one_hot)
#             else:
#                 # If no objects in this scale/batch, class loss is 0
#                 class_loss = torch.tensor(0.0, device=pred.device)
#             # --- FIX ENDS HERE ---

#             total_loss += (
#                 self.lambda_box * box_loss
#                 + self.lambda_obj * obj_loss
#                 + self.lambda_noobj * noobj_loss
#                 + self.lambda_class * class_loss
#             )
            
#         return total_loss

In [ ]:
class YoloLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        self.bce = nn.BCEWithLogitsLoss()
        self.lambda_class = 1
        self.lambda_noobj = 10
        self.lambda_obj = 1
        self.lambda_box = 10

    def forward(self, predictions, targets, anchors):
        total_loss = 0
        
        for i in range(3):
            pred = predictions[i]
            target = targets[i]
            
            # Reshape anchors for broadcasting
            scale_anchors = anchors[i].reshape(1, 3, 1, 1, 2)
            
            obj_mask = target[..., 0] == 1
            noobj_mask = target[..., 0] == 0

            # No Object Loss
            noobj_loss = self.bce(
                (pred[..., 0:1][noobj_mask]), (target[..., 0:1][noobj_mask])
            )

            # Object Loss
            obj_loss = self.bce(
                (pred[..., 0:1][obj_mask]), (target[..., 0:1][obj_mask])
            )

            # Box Coordinate Loss
            pred[..., 1:3] = torch.sigmoid(pred[..., 1:3]) # x,y
            
            # --- FIX 1: Numerical Stability for Width/Height ---
            target_wh_ratio = target[..., 3:5] / scale_anchors
            target_wh_ratio = torch.clamp(target_wh_ratio, min=1e-6)
            
            target[..., 3:5] = torch.log(target_wh_ratio) 
            
            box_loss = self.mse(pred[..., 1:5][obj_mask], target[..., 1:5][obj_mask])
            
            # Class Loss (with safe handling for empty batches)
            if obj_mask.any():
                pred_classes = pred[..., 5:][obj_mask]
                target_indices = target[..., 5][obj_mask].long()
                num_classes = pred_classes.shape[1]
                target_one_hot = torch.nn.functional.one_hot(target_indices, num_classes=num_classes).float()
                class_loss = self.bce(pred_classes, target_one_hot)
            else:
                class_loss = torch.tensor(0.0, device=pred.device)

            total_loss += (
                self.lambda_box * box_loss
                + self.lambda_obj * obj_loss
                + self.lambda_noobj * noobj_loss
                + self.lambda_class * class_loss
            )
            
        return total_loss

In [11]:
def train_model(model, loader, optimizer, criterion, device, scaled_anchors):
    model.train()
    for epoch in range(EPOCHS):
        loop = tqdm(loader, leave=True)
        total_loss = 0
        for imgs, labels in loop:
            imgs = imgs.to(device)
            labels = (
                labels[0].to(device),
                labels[1].to(device),
                labels[2].to(device),
            )
            
            preds = model(imgs)
            loss = criterion(preds, labels, scaled_anchors)
            

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)

            total_loss += loss.item()
            loop.set_description(f"Epoch {epoch+1}/{EPOCHS}")
            loop.set_postfix(loss=loss.item())
        
        print(f"Epoch {epoch+1} Average Loss: {total_loss / len(loader)}")


# def train_model(model, loader, optimizer, criterion, device, scaled_anchors):
#     model.train()
#     # Add a loop bar description
#     loop = tqdm(loader, leave=True)
#     total_loss = 0
    
#     for imgs, labels in loop:
#         imgs = imgs.to(device)
#         labels = (
#             labels[0].to(device),
#             labels[1].to(device),
#             labels[2].to(device),
#         )
        
#         optimizer.zero_grad()
        
#         preds = model(imgs)
#         loss = criterion(preds, labels, scaled_anchors)
        
#         loss.backward()
        
#         # --- FIX 2: Gradient Clipping ---
#         # This prevents the gradients from exploding, which causes NaNs
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
        
#         optimizer.step()

#         total_loss += loss.item()
        
#         loop.set_description(f"Epoch {epoch+1}/{EPOCHS}")
#         # Update tqdm to show current loss
#         loop.set_postfix(loss=loss.item())
    
#     # Print average loss for the epoch
#     print(f"Average Loss: {total_loss / len(loader)}")

In [12]:
def plot_image(image_tensor, boxes):
    # Convert tensor to PIL Image
    im = transforms.ToPILImage()(image_tensor.cpu())
    if im.mode != "RGB":
        im = im.convert("RGB")
        
    draw = ImageDraw.Draw(im)
    width, height = im.size

    for box in boxes:
        # box format: [class, conf, x, y, w, h]
        class_pred, conf, x, y, w, h = box
        
        # Convert from center format to top-left corner format
        upper_left_x = (x - w / 2) * width
        upper_left_y = (y - h / 2) * height
        lower_right_x = (x + w / 2) * width
        lower_right_y = (y + h / 2) * height

        # Draw bounding box
        draw.rectangle(
            [upper_left_x, upper_left_y, lower_right_x, lower_right_y],
            outline="red",
            width=2
        )
        
        # Draw label
        text = f"Class {int(class_pred)}: {conf:.2f}"
        text_bbox = draw.textbbox((upper_left_x, upper_left_y), text)
        draw.rectangle(text_bbox, fill="red")
        draw.text((upper_left_x, upper_left_y), text, fill="white")

    plt.imshow(im)
    plt.axis('off')
    plt.show()

In [13]:
def get_all_bboxes(loader, model, iou_threshold, confidence_threshold, anchors, device="cpu"):
    model.eval()
    train_idx = 0
    all_pred_boxes = []

    scaled_anchors = (
        torch.tensor(anchors)
        .reshape((3, 3, 2))
        .to(device)
    )

    for batch_idx, (x, y) in enumerate(tqdm(loader, desc="Getting BBoxes")):
        x = x.to(device)
        
        y = (y[0].to(device), y[1].to(device), y[2].to(device))

        with torch.no_grad():
            predictions = model(x)

        batch_size = x.shape[0]
        
        for i in range(batch_size):
            pred_boxes_single_image = []
            # For each scale
            for scale_idx in range(3):
                S = predictions[scale_idx].shape[2]
                # For each anchor on that scale
                for anchor_idx in range(3):
                    # Get all predictions where objectness is above threshold
                    obj_conf = torch.sigmoid(predictions[scale_idx][i, anchor_idx, ..., 0])
                    conf_mask = obj_conf > confidence_threshold
                    
                    if not conf_mask.any():
                        continue

                    # Extract confident predictions
                    preds_on_scale = predictions[scale_idx][i, anchor_idx, conf_mask, :]
                    grid_y, grid_x = torch.where(conf_mask)
                    
                    # Decode bounding box coordinates
                    box_coords = torch.sigmoid(preds_on_scale[:, 1:3])
                    x_center = (box_coords[:, 0] + grid_x) / S
                    y_center = (box_coords[:, 1] + grid_y) / S
                    
                    # Decode width and height
                    anchor = scaled_anchors[scale_idx, anchor_idx]
                    box_wh = torch.exp(preds_on_scale[:, 3:5]) * anchor
                    w = box_wh[:, 0] / IMG_SIZE
                    h = box_wh[:, 1] / IMG_SIZE
                    
                    # Get class predictions
                    class_probs = torch.sigmoid(preds_on_scale[:, 5:])
                    class_conf, class_label = torch.max(class_probs, dim=1)
                    
                    # Combine into [class, conf, x, y, w, h] format
                    final_conf = (torch.sigmoid(preds_on_scale[:, 0]) * class_conf).float()
                    
                    # Filter again by the final confidence
                    final_conf_mask = final_conf > confidence_threshold
                    if not final_conf_mask.any():
                        continue
                        
                    pred_boxes_batch = torch.cat([
                        class_label[final_conf_mask].float().unsqueeze(1),
                        final_conf[final_conf_mask].unsqueeze(1),
                        x_center[final_conf_mask].unsqueeze(1),
                        y_center[final_conf_mask].unsqueeze(1),
                        w[final_conf_mask].unsqueeze(1),
                        h[final_conf_mask].unsqueeze(1)
                    ], dim=1)
                    
                    pred_boxes_single_image.extend(pred_boxes_batch.tolist())

            # Run NMS on all boxes for this image
            nms_boxes = non_max_suppression(pred_boxes_single_image, iou_threshold, confidence_threshold)
            for nms_box in nms_boxes:
                all_pred_boxes.append([train_idx] + nms_box)
            train_idx += 1
            
    model.train()
    return all_pred_boxes

In [14]:
def mean_average_precision(pred_boxes, true_boxes, iou_threshold=0.5, num_classes=11):
    average_precisions = []

    for c in range(num_classes):
        detections = [d for d in pred_boxes if d[1] == c]
        ground_truths = [gt for gt in true_boxes if gt[1] == c]

        # Count how many ground truth boxes are in each image
        amount_bboxes = Counter(gt[0] for gt in ground_truths)
        for key, val in amount_bboxes.items():
            amount_bboxes[key] = torch.zeros(val)

        # Sort detections by confidence
        detections.sort(key=lambda x: x[2], reverse=True)
        TP = torch.zeros(len(detections))
        FP = torch.zeros(len(detections))
        total_true_bboxes = len(ground_truths)
        
        if total_true_bboxes == 0:
            continue

        for detection_idx, detection in enumerate(detections):
            # Get all ground truth boxes for the same image as the detection
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detection[0]
            ]

            best_iou = 0
            best_gt_idx = -1

            for idx, gt in enumerate(ground_truth_img):
                iou = iou_boxes(torch.tensor(detection[1:]), torch.tensor(gt[1:]))
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                # Check if we haven't already matched this ground truth box
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    TP[detection_idx] = 1 # Mark as True Positive
                    amount_bboxes[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1 # It's a duplicate detection
            else:
                FP[detection_idx] = 1 # Failed to meet IoU threshold

        # Calculate precision and recall
        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        recalls = TP_cumsum / (total_true_bboxes)
        precisions = TP_cumsum / (TP_cumsum + FP_cumsum)
        
        # Integrate under the precision-recall curve
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        average_precisions.append(torch.trapz(precisions, recalls))

    return sum(average_precisions) / (len(average_precisions))

In [15]:
def check_accuracy(loader, model, device):
    print("\nCalculating mAP on dataset ")
    model.to(device)
    pred_boxes = get_all_bboxes(
        loader, model, 
        iou_threshold=IOU_THRESHOLD, 
        confidence_threshold=CONFIDENCE_THRESHOLD, 
        anchors=ANCHORS, 
        device=device
    )
    
    map_val = mean_average_precision(pred_boxes, iou_threshold=IOU_THRESHOLD, num_classes=NUM_CLASSES)
    print(f"mAP: {map_val:.4f}")
    return map_val

# Training on the dataset 

In [16]:
# if __name__ == '__main__':
    
#     print("Training on 1T dataset: ")
#     # training on the 1T dataset 
#     model_1T, class_names,test_image_dir_1T,test_label_dir_1T, train_losses_1T, val_losses_1T, mAPs_1T, mAP50s_1T, mAP75s_1T,train_loader_1T= train(
#         base_dir=r"/home/abdullah/YOLO_BTP/RadDet_128_1T/RadDet40k128HW001Tv2",
#         num_epochs=100,
#         save_name="yolov1_1T_with_and_nms_threshold_0.5_and_epoch100_and_patience_35_lr1e4_mAP_27thOct.pt"
#     )


if __name__ == "__main__":
    image_dir = "/home/abdullah/YOLO_BTP/RadDet_128_1T/RadDet40k128HW001Tv2/images/train"
    label_dir = "/home/abdullah/YOLO_BTP/RadDet_128_1T/RadDet40k128HW001Tv2/labels/train"
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Anchors for loss calculation 
    scaled_anchors = (
        torch.tensor(ANCHORS) / torch.tensor([IMG_SIZE, IMG_SIZE])
    ).to(device)

    dataset = RadarDataset(image_dir, label_dir, anchors=ANCHORS, S=S, C=NUM_CLASSES)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)

    model = YOLOv3(in_channels=1, num_classes=NUM_CLASSES).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    criterion = YoloLoss()
    
    for epoch in range(EPOCHS):
        train_model(model, loader, optimizer, criterion, device, scaled_anchors)
        check_accuracy(loader, model, device)
    
    print("\nFinal Evaluation: ")
    check_accuracy(loader, model, device)
    
    print("\nVisualizing a sample prediction: ")
    model.eval()
    x, y = next(iter(loader))
    x = x.to(device)
    with torch.no_grad():
        out = model(x)
    
    all_preds = get_all_bboxes(
        [(x, y)], model, IOU_THRESHOLD, CONFIDENCE_THRESHOLD, ANCHORS, device
    )
    
    # Filter boxes for the first image in the batch
    boxes_for_image_0 = [box[1:] for box in all_preds if box[0] == 0]
    
    plot_image(x[0], boxes_for_image_0)

Using device: cuda


Epoch 1/50: 100%|██████████| 1751/1751 [17:26<00:00,  1.67it/s, loss=nan]


Epoch 1 Average Loss: nan


Epoch 2/50: 100%|██████████| 1751/1751 [17:25<00:00,  1.68it/s, loss=nan]


Epoch 2 Average Loss: nan


...

Epoch 49 Average Loss: nan


Epoch 50/50: 100%|██████████| 1751/1751 [16:35<00:00,  1.76it/s, loss=nan]


Epoch 50 Average Loss: nan

Calculating mAP on dataset 


Getting BBoxes: 100%|██████████| 1751/1751 [09:41<00:00,  3.01it/s]


TypeError: mean_average_precision() missing 1 required positional argument: 'true_boxes'

## rough

In [ ]:
# import os
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
# from PIL import Image, ImageDraw
# import numpy as np
# from tqdm import tqdm
# import glob
# from collections import Counter
# import matplotlib.pyplot as plt

# # --- HYPERPARAMETERS ---
# IMG_SIZE = 416
# BATCH_SIZE = 8
# LEARNING_RATE = 1e-4 # Slightly increased for stability with Adam
# EPOCHS = 50 
# NUM_CLASSES = 11
# CONFIDENCE_THRESHOLD = 0.5
# IOU_THRESHOLD = 0.5
# ANCHORS = [
#     [(116, 90), (156, 198), (373, 326)],  # Scale 1 (13x13)
#     [(30, 61), (62, 45), (59, 119)],      # Scale 2 (26x26)
#     [(10, 13), (16, 30), (33, 23)],       # Scale 3 (52x52)
# ]
# S = [IMG_SIZE // 32, IMG_SIZE // 16, IMG_SIZE // 8]

# # --- MODEL COMPONENTS ---
# class CNNBlock(nn.Module):
#     def __init__(self, in_channels, out_channels, use_bn=True, **kwargs):
#         super().__init__()
#         self.conv = nn.Conv2d(in_channels, out_channels, bias=not use_bn, **kwargs)
#         self.bn = nn.BatchNorm2d(out_channels) if use_bn else nn.Identity()
#         self.leaky = nn.LeakyReLU(0.1)

#     def forward(self, x):
#         return self.leaky(self.bn(self.conv(x)))

# class ResidualBlock(nn.Module):
#     def __init__(self, channels, num_repeats=1):
#         super().__init__()
#         self.layers = nn.ModuleList()
#         for _ in range(num_repeats):
#             self.layers += [
#                 nn.Sequential(
#                     CNNBlock(channels, channels // 2, kernel_size=1),
#                     CNNBlock(channels // 2, channels, kernel_size=3, padding=1),
#                 )
#             ]

#     def forward(self, x):
#         for layer in self.layers:
#             x = x + layer(x)
#         return x

# class PredictionHead(nn.Module):
#     def __init__(self, in_channels, num_classes, anchors):
#         super().__init__()
#         self.num_classes = num_classes
#         self.anchors = anchors
#         self.num_anchors = len(anchors)
        
#         self.head = nn.Sequential(
#             CNNBlock(in_channels, in_channels * 2, kernel_size=3, padding=1),
#             CNNBlock(in_channels * 2, (self.num_anchors * (5 + num_classes)), use_bn=False, kernel_size=1),
#         )

#     def forward(self, x):
#         out = self.head(x)
#         out = out.view(x.shape[0], self.num_anchors, 5 + self.num_classes, x.shape[2], x.shape[3])
#         out = out.permute(0, 1, 3, 4, 2)
#         return out

# class Darknet53(nn.Module):
#     def __init__(self, in_channels=1):
#         super().__init__()
#         self.in_channels = in_channels
#         self.layers = nn.ModuleList([
#             CNNBlock(in_channels, 32, kernel_size=3, padding=1),
#             CNNBlock(32, 64, kernel_size=3, padding=1, stride=2),
#             ResidualBlock(64, num_repeats=1),
#             CNNBlock(64, 128, kernel_size=3, padding=1, stride=2),
#             ResidualBlock(128, num_repeats=2),
#             CNNBlock(128, 256, kernel_size=3, padding=1, stride=2),
#             ResidualBlock(256, num_repeats=8),
#             CNNBlock(256, 512, kernel_size=3, padding=1, stride=2),
#             ResidualBlock(512, num_repeats=8),
#             CNNBlock(512, 1024, kernel_size=3, padding=1, stride=2),
#             ResidualBlock(1024, num_repeats=4),
#         ])

#     def forward(self, x):
#         outputs = []
#         for i, layer in enumerate(self.layers):
#             x = layer(x)
#             if i in [6, 8]:
#                 outputs.append(x)
#         outputs.append(x)
#         return outputs[0], outputs[1], outputs[2]

# class YOLOv3(nn.Module):
#     def __init__(self, in_channels=1, num_classes=11):
#         super().__init__()
#         self.num_classes = num_classes
#         self.in_channels = in_channels
#         self.backbone = Darknet53(in_channels=in_channels)
        
#         # FIXED: Corrected input channels for heads and convs
#         self.head1 = PredictionHead(1024, num_classes, ANCHORS[0]) 
#         self.head2 = PredictionHead(1024, num_classes, ANCHORS[1]) # Input 1024 (512+512)
#         self.head3 = PredictionHead(512, num_classes, ANCHORS[2])  # Input 512 (256+256)

#         self.conv1 = CNNBlock(1024, 512, kernel_size=1)
#         self.conv2 = CNNBlock(1024, 256, kernel_size=1) # Input 1024
#         self.upsample = nn.Upsample(scale_factor=2, mode="nearest")

#     def forward(self, x):
#         route3, route2, route1 = self.backbone(x)
        
#         out1 = self.head1(route1)
        
#         x = self.conv1(route1)
#         x = self.upsample(x)
#         x = torch.cat([x, route2], dim=1)
#         out2 = self.head2(x)

#         x = self.conv2(x)
#         x = self.upsample(x)
#         x = torch.cat([x, route3], dim=1)
#         out3 = self.head3(x)

#         return out1, out2, out3

# # --- UTILS ---
# def iou_boxes(box1, box2, box_format="xywh"):
#     eps=1e-6
#     if box_format == "xywh":
#         box1_x1 = box1[..., 0:1] - box1[..., 2:3] / 2
#         box1_y1 = box1[..., 1:2] - box1[..., 3:4] / 2
#         box1_x2 = box1[..., 0:1] + box1[..., 2:3] / 2
#         box1_y2 = box1[..., 1:2] + box1[..., 3:4] / 2
#         box2_x1 = box2[..., 0:1] - box2[..., 2:3] / 2
#         box2_y1 = box2[..., 1:2] - box2[..., 3:4] / 2
#         box2_x2 = box2[..., 0:1] + box2[..., 2:3] / 2
#         box2_y2 = box2[..., 1:2] + box2[..., 3:4] / 2
#     else:
#         box1_x1, box1_y1, box1_x2, box1_y2 = box1[..., 0:1], box1[..., 1:2], box1[..., 2:3], box1[..., 3:4]
#         box2_x1, box2_y1, box2_x2, box2_y2 = box2[..., 0:1], box2[..., 1:2], box2[..., 2:3], box2[..., 3:4]

#     x1 = torch.max(box1_x1, box2_x1)
#     y1 = torch.max(box1_y1, box2_y1)
#     x2 = torch.min(box1_x2, box2_x2)
#     y2 = torch.min(box1_y2, box2_y2)

#     inter_w = (x2 - x1).clamp(min=0)
#     inter_h = (y2 - y1).clamp(min=0)
#     intersection = inter_w * inter_h 

#     area1 = abs((box1_x2 - box1_x1).clamp(min=0) * (box1_y2 - box1_y1).clamp(min=0))
#     area2 = abs((box2_x2 - box2_x1).clamp(min=0) * (box2_y2 - box2_y1).clamp(min=0))

#     union = area1 + area2 - intersection
#     return intersection / (union+eps)

# def non_max_suppression(bboxes, iou_threshold, confidence_threshold):
#     bboxes = [box for box in bboxes if box[1] > confidence_threshold]
#     bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
#     bboxes_after_nms = []

#     while bboxes:
#         chosen_box = bboxes.pop(0)
#         bboxes = [
#             box for box in bboxes
#             if box[0] != chosen_box[0] or 
#             iou_boxes(torch.tensor(chosen_box[2:]), torch.tensor(box[2:])) < iou_threshold
#         ]
#         bboxes_after_nms.append(chosen_box)
#     return bboxes_after_nms

# # --- DATASET ---
# class RadarDataset(Dataset):
#     def __init__(self, image_dir, label_dir, anchors, S, C=1):
#         self.image_paths = sorted(glob.glob(os.path.join(image_dir, '*.png')))
#         self.label_dir = label_dir
#         self.S = S
#         self.C = C
#         self.anchors = torch.tensor(anchors[0] + anchors[1] + anchors[2])
#         self.num_anchors = self.anchors.shape[0]
#         self.num_anchors_per_scale = self.num_anchors // 3
        
#         self.transform = transforms.Compose([
#             transforms.Grayscale(),
#             transforms.Resize((IMG_SIZE, IMG_SIZE)),
#             transforms.ToTensor()
#         ])

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         image_path = self.image_paths[idx]
#         label_path = os.path.join(self.label_dir, os.path.basename(image_path).replace('.png', '.txt'))
#         image = Image.open(image_path)
#         image = self.transform(image)

#         targets = [torch.zeros((self.num_anchors_per_scale, s, s, 6)) for s in self.S]

#         if os.path.exists(label_path):
#             with open(label_path, 'r') as f:
#                 for line in f:
#                     data = list(map(float, line.strip().split()))
#                     if len(data) < 5: continue
#                     cls, x, y, w, h = data
                    
#                     # FIXED: Padding for IoU calculation
#                     box_tensor = torch.tensor([0, 0, w, h])
#                     anchors_xywh = torch.cat([torch.zeros_like(self.anchors), self.anchors], dim=1)
#                     ious = iou_boxes(box_tensor, anchors_xywh)
#                     best_anchor_idx = ious.argmax()
                    
#                     scale_idx = best_anchor_idx // self.num_anchors_per_scale
#                     anchor_on_scale_idx = best_anchor_idx % self.num_anchors_per_scale
                    
#                     s = self.S[scale_idx]
#                     # FIXED: Clamping indices to avoid out of bounds
#                     i, j = int(s * y), int(s * x)
#                     i = min(i, s - 1)
#                     j = min(j, s - 1)
                    
#                     if targets[scale_idx][anchor_on_scale_idx, i, j, 0] == 0:
#                         targets[scale_idx][anchor_on_scale_idx, i, j, 0] = 1 
#                         x_cell, y_cell = s * x - j, s * y - i
#                         w_cell, h_cell = w, h
#                         box_coords = torch.tensor([x_cell, y_cell, w_cell, h_cell])
#                         targets[scale_idx][anchor_on_scale_idx, i, j, 1:5] = box_coords
#                         targets[scale_idx][anchor_on_scale_idx, i, j, 5] = int(cls)
#         return image, tuple(targets)

# # --- LOSS FUNCTION ---
# class YoloLoss(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.mse = nn.MSELoss()
#         self.bce = nn.BCEWithLogitsLoss()
#         self.lambda_class = 1
#         self.lambda_noobj = 10
#         self.lambda_obj = 1
#         self.lambda_box = 10

#     def forward(self, predictions, targets, anchors):
#         total_loss = 0
        
#         for i in range(3):
#             pred = predictions[i]
#             target = targets[i]
            
#             # Reshape anchors: (1, 3, 1, 1, 2)
#             scale_anchors = anchors[i].reshape(1, 3, 1, 1, 2)
            
#             obj_mask = target[..., 0] == 1
#             noobj_mask = target[..., 0] == 0

#             # --- No Object Loss ---
#             noobj_loss = self.bce(
#                 (pred[..., 0:1][noobj_mask]), (target[..., 0:1][noobj_mask])
#             )

#             # --- Object Loss ---
#             obj_loss = self.bce(
#                 (pred[..., 0:1][obj_mask]), (target[..., 0:1][obj_mask])
#             )

#             # --- Box Coordinates Loss ---
#             # FIXED: Calculate loss ONLY on the masked valid objects to avoid log(0)
#             if obj_mask.any():
#                 # Extract valid predictions and targets
#                 pred_boxes = pred[..., 1:5][obj_mask]
#                 target_boxes = target[..., 1:5][obj_mask]
#                 anchor_boxes = scale_anchors.expand_as(target[..., 3:5])[obj_mask]
                
#                 # Sigmoid for x, y
#                 pred_xy = torch.sigmoid(pred_boxes[..., 0:2])
#                 target_xy = target_boxes[..., 0:2]
                
#                 # Process w, h
#                 # Instead of mutating target in place, we calculate the value needed for MSE
#                 # Target for network output is log(target_w / anchor_w)
#                 target_wh = torch.log((target_boxes[..., 2:4] / anchor_boxes) + 1e-6)
#                 pred_wh = pred_boxes[..., 2:4]

#                 box_loss = self.mse(pred_xy, target_xy) + self.mse(pred_wh, target_wh)
#             else:
#                 box_loss = torch.tensor(0.0, device=pred.device)

#             # --- Class Loss ---
#             if obj_mask.any():
#                 pred_classes = pred[..., 5:][obj_mask]
#                 target_indices = target[..., 5][obj_mask].long()
                
#                 # One-hot encode
#                 num_classes = pred_classes.shape[1]
#                 target_one_hot = torch.nn.functional.one_hot(target_indices, num_classes=num_classes).float()
                
#                 class_loss = self.bce(pred_classes, target_one_hot)
#             else:
#                 class_loss = torch.tensor(0.0, device=pred.device)

#             total_loss += (
#                 self.lambda_box * box_loss
#                 + self.lambda_obj * obj_loss
#                 + self.lambda_noobj * noobj_loss
#                 + self.lambda_class * class_loss
#             )
            
#         return total_loss

# # --- EVALUATION ---
# def get_all_bboxes(loader, model, iou_threshold, confidence_threshold, anchors, device="cpu"):
#     model.eval()
#     train_idx = 0
#     all_pred_boxes = []
    
#     scaled_anchors = (torch.tensor(anchors).reshape((3, 3, 2)).to(device))

#     for batch_idx, (x, y) in enumerate(tqdm(loader, desc="Getting BBoxes", leave=False)):
#         x = x.to(device)
#         with torch.no_grad():
#             predictions = model(x)

#         batch_size = x.shape[0]
#         for i in range(batch_size):
#             pred_boxes_single_image = []
#             for scale_idx in range(3):
#                 S = predictions[scale_idx].shape[2]
#                 for anchor_idx in range(3):
#                     obj_conf = torch.sigmoid(predictions[scale_idx][i, anchor_idx, ..., 0])
#                     conf_mask = obj_conf > confidence_threshold
                    
#                     if not conf_mask.any(): continue

#                     preds_on_scale = predictions[scale_idx][i, anchor_idx, conf_mask, :]
#                     grid_y, grid_x = torch.where(conf_mask)
                    
#                     box_coords = torch.sigmoid(preds_on_scale[:, 1:3])
#                     x_center = (box_coords[:, 0] + grid_x) / S
#                     y_center = (box_coords[:, 1] + grid_y) / S
                    
#                     anchor = scaled_anchors[scale_idx, anchor_idx]
#                     # FIXED: clamp exp to avoid infinity
#                     box_wh_preds = torch.clamp(preds_on_scale[:, 3:5], max=10) 
#                     box_wh = torch.exp(box_wh_preds) * anchor
#                     w = box_wh[:, 0] / IMG_SIZE
#                     h = box_wh[:, 1] / IMG_SIZE
                    
#                     class_probs = torch.sigmoid(preds_on_scale[:, 5:])
#                     class_conf, class_label = torch.max(class_probs, dim=1)
#                     final_conf = (torch.sigmoid(preds_on_scale[:, 0]) * class_conf).float()
                    
#                     final_conf_mask = final_conf > confidence_threshold
#                     if not final_conf_mask.any(): continue
                        
#                     pred_boxes_batch = torch.cat([
#                         class_label[final_conf_mask].float().unsqueeze(1),
#                         final_conf[final_conf_mask].unsqueeze(1),
#                         x_center[final_conf_mask].unsqueeze(1),
#                         y_center[final_conf_mask].unsqueeze(1),
#                         w[final_conf_mask].unsqueeze(1),
#                         h[final_conf_mask].unsqueeze(1)
#                     ], dim=1)
#                     pred_boxes_single_image.extend(pred_boxes_batch.tolist())

#             nms_boxes = non_max_suppression(pred_boxes_single_image, iou_threshold, confidence_threshold)
#             for nms_box in nms_boxes:
#                 all_pred_boxes.append([train_idx] + nms_box)
#             train_idx += 1
            
#     model.train()
#     return all_pred_boxes

# def mean_average_precision(pred_boxes, true_boxes, iou_threshold=0.5, num_classes=11):
#     average_precisions = []
#     epsilon = 1e-6

#     for c in range(num_classes):
#         detections = [d for d in pred_boxes if d[1] == c]
#         ground_truths = [gt for gt in true_boxes if gt[1] == c]

#         amount_bboxes = Counter(gt[0] for gt in ground_truths)
#         for key, val in amount_bboxes.items():
#             amount_bboxes[key] = torch.zeros(val)

#         detections.sort(key=lambda x: x[2], reverse=True)
#         TP = torch.zeros(len(detections))
#         FP = torch.zeros(len(detections))
#         total_true_bboxes = len(ground_truths)
        
#         if total_true_bboxes == 0: continue

#         for detection_idx, detection in enumerate(detections):
#             ground_truth_img = [bbox for bbox in ground_truths if bbox[0] == detection[0]]
#             best_iou = 0
#             best_gt_idx = -1

#             for idx, gt in enumerate(ground_truth_img):
#                 iou = iou_boxes(torch.tensor(detection[1:]), torch.tensor(gt[1:]))
#                 if iou > best_iou:
#                     best_iou = iou
#                     best_gt_idx = idx

#             if best_iou > iou_threshold:
#                 if amount_bboxes[detection[0]][best_gt_idx] == 0:
#                     TP[detection_idx] = 1
#                     amount_bboxes[detection[0]][best_gt_idx] = 1
#                 else:
#                     FP[detection_idx] = 1
#             else:
#                 FP[detection_idx] = 1

#         TP_cumsum = torch.cumsum(TP, dim=0)
#         FP_cumsum = torch.cumsum(FP, dim=0)
#         recalls = TP_cumsum / (total_true_bboxes + epsilon)
#         precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)
        
#         precisions = torch.cat((torch.tensor([1]), precisions))
#         recalls = torch.cat((torch.tensor([0]), recalls))
#         average_precisions.append(torch.trapz(precisions, recalls))

#     return sum(average_precisions) / (len(average_precisions) + epsilon)

# def check_accuracy(loader, model, device):
#     print("\nCalculating mAP...")
#     model.eval()
    
#     # Get predictions
#     pred_boxes = get_all_bboxes(
#         loader, model, 
#         iou_threshold=IOU_THRESHOLD, 
#         confidence_threshold=CONFIDENCE_THRESHOLD, 
#         anchors=ANCHORS, 
#         device=device
#     )
    
#     # Get ground truths (simplified extraction from dataset)
#     true_boxes = []
#     for idx in range(len(loader.dataset)):
#         _, targets = loader.dataset[idx]
#         for scale_idx in range(3):
#             anchors_on_scale = targets[scale_idx]
#             # Mask where object exists: [3, S, S, 6]
#             mask = anchors_on_scale[..., 0] == 1
#             if mask.any():
#                 # Extract: [class, x, y, w, h]
#                 # targets storage: [obj, x, y, w, h, class]
#                 valid_targets = anchors_on_scale[mask]
#                 for t in valid_targets:
#                     # Format for mAP: [train_idx, class, prob(1), x, y, w, h]
#                     # GT prob is 1.0
#                     true_boxes.append([
#                         idx, 
#                         t[5].item(), 
#                         1.0, 
#                         t[1].item(), t[2].item(), t[3].item(), t[4].item()
#                     ])

#     map_val = mean_average_precision(pred_boxes, true_boxes, iou_threshold=IOU_THRESHOLD, num_classes=NUM_CLASSES)
#     print(f"mAP: {map_val:.4f}")
#     model.train()
#     return map_val

# def train_model(model, loader, optimizer, criterion, device, scaled_anchors):
#     model.train()
#     loop = tqdm(loader, leave=True)
#     total_loss = 0
    
#     for imgs, labels in loop:
#         imgs = imgs.to(device)
#         labels = (
#             labels[0].to(device),
#             labels[1].to(device),
#             labels[2].to(device),
#         )
        
#         optimizer.zero_grad()
#         preds = model(imgs)
        
#         # Check for NaN in inputs
#         if torch.isnan(imgs).any():
#             print("Warning: NaN input image detected")
#             continue

#         loss = criterion(preds, labels, scaled_anchors)
        
#         if torch.isnan(loss):
#             print("Warning: Loss is NaN, skipping step")
#             optimizer.zero_grad()
#             continue
            
#         loss.backward()
        
#         # FIXED: Gradient Clipping
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        
#         optimizer.step()

#         total_loss += loss.item()
#         loop.set_postfix(loss=loss.item())
    
#     avg_loss = total_loss / len(loader)
#     print(f"Average Loss: {avg_loss}")
#     return avg_loss

# # --- MAIN BLOCK ---
# if __name__ == "__main__":
#     # Update these paths to your actual directories
#     image_dir = "/home/abdullah/YOLO_BTP/RadDet_128_1T/RadDet40k128HW001Tv2/images/train"
#     label_dir = "/home/abdullah/YOLO_BTP/RadDet_128_1T/RadDet40k128HW001Tv2/labels/train"
    
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Using device: {device}")
    
#     # Scaled anchors for loss calculation 
#     scaled_anchors = (
#         torch.tensor(ANCHORS) / torch.tensor([IMG_SIZE, IMG_SIZE])
#     ).to(device)

#     dataset = RadarDataset(image_dir, label_dir, anchors=ANCHORS, S=S, C=NUM_CLASSES)
#     loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)

#     model = YOLOv3(in_channels=1, num_classes=NUM_CLASSES).to(device)
    
#     # Use a safe epsilon for Adam
#     optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4, eps=1e-6)
#     criterion = YoloLoss()
    
#     # Training Loop
#     for epoch in range(EPOCHS):
#         print(f"\nEpoch {epoch+1}/{EPOCHS}")
#         train_model(model, loader, optimizer, criterion, device, scaled_anchors)
        
#         # Evaluate every 5 epochs to save time, or every epoch if dataset is small
#         if (epoch + 1) % 5 == 0:
#              check_accuracy(loader, model, device)

Using device: cuda

Epoch 1/50


  0%|          | 1/1751 [00:00<11:11,  2.61it/s]

  0%|          | 2/1751 [00:00<09:19,  3.13it/s]

  0%|          | 3/1751 [00:00<07:45,  3.76it/s]

  0%|          | 4/1751 [00:01<07:45,  3.75it/s]

...

 10%|▉         | 175/1751 [00:41<05:27,  4.82it/s]

 10%|█         | 176/1751 [00:41<05:35,  4.69it/s]

 10%|█         | 177/1751 [00:41<05:43,  4.58it/s]

 10%|█         | 178/1751 [00:42<06:11,  4.24it/s]

KeyboardInterrupt: 